<a href="https://colab.research.google.com/github/morwald/ada_project/blob/main/milestone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports of libraries and additional datasets

In [1]:
# Import epfl google drive!
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Change to shared project drive
%cd /content/drive/Shareddrives/ADA-project

/content/drive/Shareddrives/ADA-project


In [3]:
import pandas as pd
import bz2
import json

In [4]:
# Speaker attributes in df
df = pd.read_parquet('Project datasets/speaker_attributes.parquet')
df.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [ ]:
!python Project\ datasets/query_wikidata_dump.py --input /content/drive/MyDrive/wikidata-20211004-all.json.gz  --output speakers.json

##Cleaning 

### Plan
*   Descriptive statistics
*   types
*   Missing data
*   distribution (histograms)
*   outliers
*   duplicates
*  inconsistent data (wrong format etc)
*   correlation between parameters
*  filter columns that we don't need
*  maybe create additional ones
*  standardize ?

In [ ]:
path_to_file = '/content/drive/MyDrive/speakers.json' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 

In [ ]:
# Loading of the quotebank dataset

path_to_file = '/content/drive/MyDrive/Quotebank/quotes-2020.json.bz2' 

with bz2.open(path_to_file, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    print(instance) 